In [1]:
from Tkinter import *
import numpy as np
import random
import sys
import time

In [2]:
class LifeGame:

    def __init__(self, L=30, rule="1 3 6/1 2", pattern=None):
        self.L = L
        # 重みづけ --- 全 半 無
        self.move_rule = [int(i) for i in rule.split("/")[0].split()]
        # 食べる幅 歩幅 ! 今回は使わない
        self.bite_rule = [int(i) for i in rule.split("/")[1].split()]

        self.lattice = np.zeros([self.L+2, self.L+2], dtype=int)
        if pattern:
            for x,y in pattern:
                self.lattice[x,y] = 2

        self.lattice[0,:] = self.lattice[self.L+1,:] = -1
        self.lattice[:,0] = self.lattice[:,self.L+1] = -1

        where = np.where(self.lattice==2)
        self.worm_site = np.array([[x,y] for x,y in zip(where[0], where[1])])
        self.worm = len(self.worm_site) # 特に使わない

    def progress(self, canvas_update, update):
        L = self.L
        Tmax = 2000
        t = 0
        move_rule = self.move_rule
        self.loop = True
        past = self.worm_site.copy()
        rn = np.random.rand
        for i, x in enumerate(self.worm_site):
            sett = True
            while sett:
                p = rn()*4
                if p < 1:   x[0] -= 1
                elif p < 2: x[1] += 1
                elif p < 3: x[0] += 1
                else:       x[1] -= 1
                if self.lattice[x[0], x[1]] != -1:
                    self.worm_site[i] = [x[0], x[1]]
                    sett = False
        self.worm_vec = self.worm_site - past
        forward = np.array([[1,0], [0,1]])
        right = np.array([[0,1], [-1,0]])
        left = np.array([[0,-1], [1,0]])
        def moveto(from_site, vec):
            return [from_site + np.dot(forward, vec),
                    from_site + np.dot(right, vec),
                    from_site + np.dot(left, vec)]
        def search(from_site, vec):
            return [from_site + np.dot(right, vec),
                    from_site + np.dot(left, vec)]
        
        while self.loop:
            try:
                past_lattice = self.lattice.copy()
                past_worm_site = self.worm_site.copy()

                # ランダムウォーク
                for i, x in enumerate(self.worm_site):
                    canvas_update(x[0],x[1],color="white")
                    box = []
                    for v in moveto(x, self.worm_vec[i]):
                        if self.lattice[v[0],v[1]] == 2:
                            for j in range(move_rule[0]):
                                box.append(v)
                        elif self.lattice[v[0],v[1]] == 1:
                            for j in range(move_rule[1]):
                                box.append(v)
                        elif self.lattice[v[0],v[1]] == 0:
                            for j in range(move_rule[2]):
                                box.append(v)
                    if box == []:
                        continue
                    new = random.choice(box)

                    # latticeの更新
                    self.lattice[new[0],new[1]] = 2
                    for v in search(new, new-x):
                        if self.lattice[v[0],v[1]] == 0:
                            self.lattice[v[0],v[1]] = 1
                            canvas_update(v[0], v[1], color="gray")
                    self.worm_vec[i] = np.array(new)-x
                    self.worm_site[i] = new

                for x in self.worm_site:
                    canvas_update(x[0], x[1], "red")
                update()
#                time.sleep(0.1)

                t += 1
                if t > Tmax:
                    self.loop = False

            except KeyboardInterrupt:
                print "stopped."
                break

In [3]:
class Draw_canvas:

    def __init__(self, lg, L):

        self.lg = lg
        self.L = L
        default_size = 640
        self.r = int(default_size/(2*self.L))
        self.fig_size = 2*self.r*self.L
        self.margin = 10
        self.sub = Toplevel()
        self.sub.title("Worm Walk")
        self.canvas = Canvas(self.sub, width=self.fig_size+2*self.margin,
                             height=self.fig_size+2*self.margin)
        self.c = self.canvas.create_rectangle
        self.update = self.canvas.update
        self.rects = dict()
        for y in range(1,self.L+1):
            for x in range(1,self.L+1):
                if self.lg.lattice[x,y] == 2:
                    live = 2
                else:
                    live = 0
                tag = (x,y)
                self.rects[tag] = Rect(x, y, live, tag, self)
        self.canvas.pack()

    def canvas_update(self, x, y, color):
        v = self.rects[(x,y)]
        v.root.canvas.itemconfig(v.ID, fill=color)

In [4]:
class Rect:
    def __init__(self, x, y, live, tag, root):
        self.root = root
        self.x = x
        self.y = y
        self.live = live
        if self.live == 2: color = "red"
        else:    color = "black"
        self.ID = self.root.c(2*(x-1)*self.root.r+self.root.margin,
                        2*(y-1)*self.root.r+self.root.margin,
                        2*x*self.root.r+self.root.margin,
                        2*y*self.root.r+self.root.margin,
                        outline="#202020", fill=color, tag=tag)
        self.root.canvas.tag_bind(self.ID, '<Button-1>', self.pressed)

    def pressed(self, event):
        if self.live == 2:
            self.live = 0
            color = "black"
            remove = set([(self.x, self.y)])
            tmp = self.root.lg.worm_site
            y = set([(tmp[i,0], tmp[i,1]) for i in range(len(tmp))])
            tmp = np.array([[i,j] for i,j in list(y - remove)])
            self.root.lg.worm_site = tmp
        else:
            self.live = 2
            color = "red"
            self.root.lg.worm_site = np.append(self.root.lg.worm_site,
                            np.array([[self.x, self.y]]), axis=0)
        self.root.lg.lattice[self.x, self.y] = self.live
        self.root.canvas.itemconfig(self.ID, fill=color)

In [5]:
class TopWindow:

    def show_window(self, title="title", *args):
        self.root = Tk()
        self.root.title(title)
        frames = []
        for i, arg in enumerate(args):
            frames.append(Frame(self.root, padx=5, pady=5))
            for k, v in arg:
                Button(frames[i],text=k,command=v).pack(expand=YES, fill='x')
            frames[i].pack(fill='x')
        self.root.mainloop()

In [6]:
class Main:

    def __init__(self):
        L = 100
        worm = 5
        rule = "1 3 10/1 2"
        self.top = TopWindow()
        pattern = [(np.random.randint(1,L+1),np.random.randint(1,L+1))
                        for i in range(worm)]

        self.lg = LifeGame(L, rule, pattern=pattern)
        self.top.show_window("Worm Walk", (('set', self.init),),
                                          (('start', self.start),
                                           ('pause', self.pause)),
                                          (('save', self.pr),),
                                          (('quit', self.quit),))

    def init(self):
        self.DrawCanvas = Draw_canvas(self.lg, self.lg.L)
    def start(self):
        self.lg.progress(self.DrawCanvas.canvas_update, self.DrawCanvas.update)
    def pause(self):
        self.lg.loop = False
    def pr(self):
        import tkFileDialog
        import os
        if self.DrawCanvas is None:
            return 1
        fTyp = [('eps file', '*eps'), ('all files', '*')]
        filename = tkFileDialog.asksaveasfilename(filetypes=fTyp,
                        initialdir=os.getcwd(), initialfile='figure_1.eps')
        if filename is None:
            return 0
        d = self.DrawCanvas.canvas.postscript(file=filename)
    def quit(self):
        self.pause()
        sys.exit()

if __name__ == '__main__':

    app = Main()


SystemExit: 

To exit: use 'exit', 'quit', or Ctrl-D.
